# API data load

In [285]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from urllib.parse import unquote
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [286]:
KEY = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
API_KEY = requests.utils.unquote(KEY)
API_KEY = unquote(KEY)

In [287]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회한다
# 서울
url_seoul_location = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams_seoul_location = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '204000046' })

In [288]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공한다.
url_gyeonggi_location = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams_gyeonggi_location = '?' + urlencode({ quote_plus('serviceKey') : API_KEY, quote_plus('routeId') : '204000046' })

In [289]:
# 버스도착정보조회 서비스
url_gyeonggi_arrive = 'http://openapi.gbis.go.kr/ws/rest/busarrivalservice'
queryParams_gyeonggi_arrive = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '204000046' })

In [290]:
print(url_seoul_location + queryParams_seoul_location)

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&busRouteId=204000046


In [291]:
print(url_gyeonggi_location + queryParams_gyeonggi_location)

http://openapi.gbis.go.kr/ws/rest/buslocationservice?serviceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&routeId=204000046


In [292]:
print(url_gyeonggi_arrive + queryParams_gyeonggi_arrive)

http://openapi.gbis.go.kr/ws/rest/busarrivalservice?ServiceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&busRouteId=204000046


In [293]:
buslocation_seoul = url_seoul_location + queryParams_seoul_location
busarrive_gyeonggi = url_gyeonggi_arrive + queryParams_gyeonggi_arrive
buslocation_gyeonggi = url_gyeonggi_location + queryParams_gyeonggi_location
#'http://openapi.gbis.go.kr/ws/rest/buslocationservice?serviceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&routeId=233000031'
#url_k + queryParams_k

location_seoul = Request(buslocation_seoul)
location_seoul.get_method = lambda: 'GET'
location_seoul = urlopen(location_seoul).read().decode('utf8')

location_gyeonggi = Request(buslocation_gyeonggi)
location_gyeonggi.get_method = lambda: 'GET'
location_gyeonggi = urlopen(location_gyeonggi).read().decode('utf8')

arrive_gyeonggi = Request(busarrive_gyeonggi)
arrive_gyeonggi.get_method = lambda: 'GET'
arrive_gyeonggi = urlopen(arrive_gyeonggi).read().decode('utf8')

In [294]:
xtree = ET.fromstring(location_gyeonggi)

rows = []
queryTime = xtree[1].find("queryTime").text

for node in xtree[2] :
    n_endBus = node.find("endBus").text
    n_lowPlate = node.find("lowPlate").text
    n_plateNo = node.find("plateNo").text
    n_plateType = node.find("plateType").text
    n_remainSeatCnt = node.find("remainSeatCnt").text
    n_routeId = node.find("routeId").text
    n_stationId = node.find("stationId").text
    n_stationSeq = node.find("stationSeq").text
        
    rows.append({"endBus":n_endBus,
                 "lowPlate":n_lowPlate,
                 "plateNo":n_plateNo,
                 "plateType":n_plateType,
                 "remainSeatCnt":n_remainSeatCnt,
                 "routeId":n_routeId,
                 "stationId":n_stationId,
                 "stationSeq":n_stationSeq})
rows.append({"queryTime":queryTime})

for x in rows :
    print(x)
    print('\n')

{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아7474', 'plateType': '3', 'remainSeatCnt': '40', 'routeId': '204000046', 'stationId': '101000005', 'stationSeq': '27'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기78아1147', 'plateType': '3', 'remainSeatCnt': '15', 'routeId': '204000046', 'stationId': '277103391', 'stationSeq': '18'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아8601', 'plateType': '3', 'remainSeatCnt': '35', 'routeId': '204000046', 'stationId': '102000069', 'stationSeq': '20'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아7361', 'plateType': '3', 'remainSeatCnt': '39', 'routeId': '204000046', 'stationId': '277104178', 'stationSeq': '40'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기70아8789', 'plateType': '3', 'remainSeatCnt': '34', 'routeId': '204000046', 'stationId': '206000002', 'stationSeq': '11'}


{'endBus': '0', 'lowPlate': '0', 'plateNo': '경기76사5089', 'plateType': '3', 'remainSeatCnt': '31', 'routeId': '204000046', 'stationId': '206000004', 's